# Keras ELMo Tutorial

This IPython Notebook contains an example how the ELMo embeddings from the paper *[Deep contextualized word representations (Peters et al., 2018)](http://arxiv.org/abs/1802.05365)* can be used for document classification. 

As the computation of the embeddings is computationally expensive, we will include it into a preprocessing step:
* We read in the dataset (here the IMDB dataset)
* Text is tokenized and truncated to a fix length
* Each text is fed as a sentence to the AllenNLP ElmoEmbedder to get a 1024 dimensional embedding for each word in the document
* These embeddings are then fed to our neural network that we train

Computing the embeddings once in the pre-processing significantly reduces the overall computational time. Otherwise, they would be computed for each epoch. However, this requires that enough memory is available, as our transformed dataset will constist of $\text{number_of_tokens} \cdot 1024$ float32 numbers.


**Note:** Our simple tokenization process ignores sentence boundaries and the complete document is fed as one single sentence to the ElmoEmbedder. As the ELMo embeddings are defined sentence wise, it would be better to first identify the sentences in a document and to process the document sentence by sentence to get the correct embeddings.

**Requirements**:
* Python 3.6 - lower versions of Python do not work
* AllenNLP 0.5.1 - to compute the ELMo representations
* Keras 2.2.0 - For the creation of BiLSTM-CNN-CRF architecture


See [https://github.com/UKPLab/elmo-bilstm-cnn-crf](https://github.com/UKPLab/elmo-bilstm-cnn-crf) how to install these requirements.

# Code

In [1]:
import keras
import os
import sys
from allennlp.commands.elmo import ElmoEmbedder
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Activation, Dropout

/home/reimers/anaconda3/envs/ELMo/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load all files from a directory into dictionaries
def load_directory_data(directory, label):
    data = []
    for file_path in os.listdir(directory):
        with open(os.path.join(directory, file_path), "r") as f:
            data.append({"text": f.read().replace("<br />", " "), "label": label})
    return data

# Load the positive and negative examples from the dataset
def load_dataset(directory):
    pos_data = load_directory_data(os.path.join(directory, "pos"), 1)
    neg_data = load_directory_data(os.path.join(directory, "neg"), 0)
    return pos_data+neg_data

# Download and process the IMDB dataset
def download_and_load_datasets(force_download=False):
    dataset = keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)

    train_data = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "train"))
    test_data = load_dataset(os.path.join(os.path.dirname(dataset), "aclImdb", "test"))

    return train_data, test_data

train_data, test_data = download_and_load_datasets()

random.shuffle(train_data)
random.shuffle(test_data)

In [3]:
#Tokenize text. Note, it would be better to first split it into sentences.
def tokenize_text(documents, max_tokens):
    for document in documents:
        document['tokens'] = keras.preprocessing.text.text_to_word_sequence(document['text'], lower=False)
        document['tokens'] = document['tokens'][0:max_tokens]
 
max_tokens = 100
tokenize_text(train_data, max_tokens)
tokenize_text(test_data, max_tokens)

In [4]:
# Lookup the ELMo embeddings for all documents (all sentences) in our dataset. Store those
# in a numpy matrix so that we must compute the ELMo embeddings only once.
def create_elmo_embeddings(elmo, documents, max_sentences = 1000):
    num_sentences = min(max_sentences, len(documents)) if max_sentences > 0 else len(documents)
    
    print("\n\n:: Lookup of "+str(num_sentences)+" ELMo representations. This takes a while ::")
    embeddings = []
    labels = []
    
    sentence_count = 0
    for documentIdx in range(num_sentences):
        document = documents[documentIdx]
        
        # Compute word embeddings for our sentence
        elmo_embedding = elmo.embed_sentence(document['tokens'])
        # Average the 3 layers return from ELMo
        avg_elmo_embedding = np.average(elmo_embedding, axis=0)
        
        #avg_elmo_embedding = randomEmbeddings(document['tokens'])
     
        
        embeddings.append(avg_elmo_embedding)        
        labels.append(document['label'])
        
        
        # Some progress info
        sentence_count += 1
        percent = 100.0 * sentence_count / num_sentences
        line = '[{0}{1}]'.format('=' * int(percent / 2), ' ' * (50 - int(percent / 2)))
        status = '\r{0:3.0f}%{1} {2:3d}/{3:3d} sentences'
        sys.stdout.write(status.format(percent, line, sentence_count, num_sentences))
        
        if max_sentences > 0 and sentence_count > max_sentences:
            break
        
    return embeddings, labels


elmo = ElmoEmbedder(cuda_device=-1) #Set cuda_device to the ID of your GPU if you have one
train_x, train_y = create_elmo_embeddings(elmo, train_data, 1000)
test_x, test_y  = create_elmo_embeddings(elmo, test_data, 1000)



:: Lookup of 1000 ELMo representations. This takes a while ::
100%[==================================================] 1000/1000 sentences

:: Lookup of 1000 ELMo representations. This takes a while ::
100%[==================================================] 1000/1000 sentences

In [5]:
# :: Pad the x matrix to uniform length ::
def pad_x_matrix(x_matrix):
    for sentenceIdx in range(len(x_matrix)):
        sent = x_matrix[sentenceIdx]
        sentence_vec = np.array(sent, dtype=np.float32)
        padding_length = max_tokens - sentence_vec.shape[0]
        if padding_length > 0:
            x_matrix[sentenceIdx] = np.append(sent, np.zeros((padding_length, sentence_vec.shape[1])), axis=0)

    matrix = np.array(x_matrix, dtype=np.float32)
    return matrix

train_x = pad_x_matrix(train_x)
train_y = np.array(train_y)

test_x = pad_x_matrix(test_x)
test_y = np.array(test_y)

print("Shape Train X:", train_x.shape)
print("Shape Test Y:", test_x.shape)

Shape Train X: (1000, 100, 1024)
Shape Test Y: (1000, 100, 1024)


In [6]:
# Simple model for sentence / document classification using CNN + global max pooling
model = Sequential()
model.add(Conv1D(filters=250, kernel_size=3, padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=10, batch_size=32)



Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.7364 - acc: 0.6200 - val_loss: 0.5560 - val_acc: 0.7090
Epoch 2/10
1000/1000 [==============================] - 1s 565us/step - loss: 0.3613 - acc: 0.8500 - val_loss: 0.4836 - val_acc: 0.7730
Epoch 3/10
1000/1000 [==============================] - 1s 569us/step - loss: 0.1361 - acc: 0.9720 - val_loss: 0.4548 - val_acc: 0.7890
Epoch 4/10
1000/1000 [==============================] - 1s 580us/step - loss: 0.0256 - acc: 1.0000 - val_loss: 0.4939 - val_acc: 0.7810
Epoch 5/10
1000/1000 [==============================] - 1s 564us/step - loss: 0.0056 - acc: 1.0000 - val_loss: 0.5270 - val_acc: 0.7830
Epoch 6/10
1000/1000 [==============================] - 1s 556us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.5359 - val_acc: 0.7810
Epoch 7/10
1000/1000 [==============================] - 1s 564us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.5495 - val_acc: 0.7820
Ep

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 100, 250)          768250    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 251       
Total params: 831,251
Trainable params: 831,251
Non-trainable params: 0
_________________________________________________________________
